# Monte Carlo simulation
The goal is to calculate the expected value of a function $f(\cdot)$ of a random variable $x$, where the distribution of $x$ is given by the probability density $\psi(x)$, i.e.
$$
E[f(x)]=\int_{-\infty}^{\infty}f(x)\psi(x)dx
$$

## Outline of the Monte Carlo simulation
1. Establish a procedure for drawing variates $x$ from the target distribution $\psi(x)$.
2. Initialise the variables:
   RunningSum = 0 
   RunningSumSquared = 0 
   $i=1$
3. Draw a realisation $x_i$ from the target distribution.
4. Add $f(x_i)$ to RunningSum and  $(f(x_i))^2$ to RunningSumSquared.
5. Increment the counter $i$. If $i$ is less than the maximum number of iterations, go to step 3.
6. Calculate the simulated mean by dividing RunningSum by the total number of iterations.
7. Calculate the variance of the simulations by dividing RunningSumSquared by the total number of iterations and subtracting the square of the mean.

## Error estimation for Monte Carlo methods
By the Central Limit Theorem, we know that for a large number $N$ of simulations, the simulation mean $X_N$ is approximately normally distributed, with standard deviation
$$
\sqrt{\frac{\sigma^2}N}
$$
where the simulation variance is an estimate for $\sigma^2$.

Thus, if there is no bias, the simulation mean is normally distributed around the target value with a standard deviation, which decreases with $\sqrt{N}$.

A 95% confidence interval for the target value is therefore approximately given by
$$
\left[X_N-2\sqrt{\frac{\sigma^2}N};X_N+2\sqrt{\frac{\sigma^2}N}\right]
$$

<font color='red'>**Monte Carlo simulation without error bounds is meaningless!**</font>

The NumPy function `random.standard_normal()` returns a random variate drawn from the standard normal distribution, while `random.standard_normal(n)` returns `n` such variates in a Numpy array:

In [ ]:
import numpy as np
print(np.random.standard_normal())
n = 5
print(np.random.standard_normal(n))

Recall that a standard normal random variable can be converted into a normal random variable of desired mean and standard deviation by multiplying by the standard deviation and adding the mean.

## Monte Carlo pricing of a Black/Scholes call option
In the Black/Scholes model, the price of the underlying stock follows Geometric Brownian motion, with the dynamics under the risk-neutral measure given by
$$S(T)=S(t)\exp\left\{\left(r−\frac12\sigma^2\right)(T−t)+\sigma(W(T)−W(t))\right\}$$
Recall that the time 0 price of a European call option (and analogously the put option) expiring at time $T$ with strike price $K$ can be expressed as the expectation under the risk-neutral measure of 
$$C=E\left[e^{−rT}\max(0,S(T)−K)\right]$$
Thus we can write a Python function which calculates the Monte Carlo estimate `MC` for the Black/Scholes price of the option and the standard deviation `MCstd` of the simulation mean, where the function takes seven arguments (in this order): $S$, $K$, $\sigma$, $r$, $T$, a 1 for a call or -1 for a put, and $n$, the number of sampling iterations of the Monte Carlo algorithm:

In [ ]:
import numpy as np
def BlackScholesMC(S,K,sgm,r,T,callput,n):
    w = np.random.standard_normal(n)
    ST=S*np.exp((r-0.5*sgm**2)*T+sgm*np.sqrt(T)*w)
    payoff=callput*(ST-K) 
    payoff=payoff*(payoff>0)
    MC=np.exp(-r*T)*np.mean(payoff)
    MCstd=np.exp(-r*T)*np.std(payoff)/np.sqrt(n)
    return MC, MCstd

To run this code with user inputs:

In [ ]:
stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
sigma = float(input('Enter the volatility: '))
interest = float(input('Enter continuously compounded interest rate: '))
maturity = float(input('Enter the time to maturity: '))
callput = int(input('Enter 1 for call or -1 for put option: '))
n = int(input('Enter the number of simulations: '))
MC, MCstd = BlackScholesMC(stock,strike,sigma,interest,maturity,callput,n)
print('The MC estimate for the option price is: ')
print(MC)
print('The 2 standard deviation confidence interval for the option price is: ')
print(MC-2*MCstd,MC+2*MCstd)